In [2]:
# Importar librerías
import numpy as np

Definir funciones

In [4]:
# Función que verifica si dos vectores son ortogonales de acuerdo a cierto nivel de tolerancia
def verificar_ortogonalidad(u,v,tol = 10**-8):
    if np.linalg.norm(u) <= tol or np.linalg.norm(v) <= tol:
        resultado = 1
    elif ((np.dot(u,v))/(np.linalg.norm(u)*np.linalg.norm(v))) <= tol:
        resultado = 1
    else:
        resultado = 0
    return resultado

# Función signo
def signo(x):
    if x<0:
        sig = -1
    else:
        sig = 1
    return sig



def svd_jacobi_aprox(A,TOL,maxsweep):
    # Calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representa a la matriz U(nxm),el segundo a la matriz S(mxm) matriz diagonal
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    # Nota: Esta función estima la SVD thin,la cual calcula unicamente las m columnas de U correspondientes a los m renglones de V. De esta
    # manera las columnas restantes de U no son calculadas, provocando una mejora significativa en velocidad de ejecución comparada con la 
    # la Full SVD. Referencia: https://en.wikipedia.org/wiki/Singular_value_decomposition#Thin_SVD.
    
    #dimensiones de A
    n = A.shape[1] #numero de columnas
    m = A.shape[0] #numero de filas
    nmax =n*(n-1)/2
    
    #inicializa valores del ciclo
    ak = A
    vk = np.identity(n, dtype = float) 
    sig = ''
    uk = ak
    num_col_ortogonal = 0
    k = 0
    stop = False
        
    
    while(k<=maxsweep & num_col_ortogonal<nmax):
        num_col_ortogonal =0
        for i in range(n-1):
            for j in range(i+1,n):
                col_j = ak[:,j]
                col_i = ak[:,i]

                #comprueba ortogonalidad  
                if(verificar_ortogonalidad(col_i,col_j,TOL)==1):
                    num_col_ortogonal =num_col_ortogonal+1
                else:
                    #calcula coeficientes de la matriz
                    a = np.dot(col_i,col_i)
                    b = np.dot(col_j,col_j)
                    c = np.dot(col_i,col_j)

                    #si c es cercano a cero no actualiza
                    if(c<TOL):
                        stop =True
                        break

                    #calcula la rotacion givens que diagonaliza
                    epsilon = (b-a)/(2*c)
                    t = signo(epsilon)/(abs(epsilon)+np.sqrt(1+epsilon**2))
                    cs = 1/np.sqrt(1+t**2)
                    sn = cs*t

                    #actualiza las columnas de la matriz ak
                    temp1 = ak[:,i].copy()
                    ak[:,i] = cs*temp1-sn*ak[:,j]
                    ak[:,j] = sn*temp1+cs*ak[:,j]


                    #actualiza las columnas de la matriz vk
                    temp2 = vk[:,i].copy() 
                    vk[:,i] = cs*temp2-sn*vk[:,j]
                    vk[:,j] = sn*temp2+cs*vk[:,j]             
                #cierra else
            #cierra for j
            if(stop):
                stop = False
                break
            
        #cierra for i
        k = k+1
     #cierra while
    
      
    #Obtener sigma (normas euclidianas de columnas de ak)
    sig = np.linalg.norm(ak, axis=0)

    #Obtener U (columnas normalizadas de ak)
    for i in range(n):
        if (sig[i]<TOL):
            uk[:,i] = 0  
        else:
            uk[:,i] = ak[:,i]/sig[i]
        

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    #index <- order(sig,decreasing = TRUE)
    index = np.argsort(sig) # Obtenemos los indices de sig ordenado
    index = index[::-1] # Invertimos, para obtener los índices del orden decreciente
    #Reordenamos
    V = vk[:,index]
    s = sig[index]
    U = uk[:,index]
    
    return U, s, V  



Definir matriz de prueba

In [7]:
np.random.seed(2020)
a = np.random.randn(9, 6)
print(a)

[[-1.76884571  0.07555227 -1.1306297  -0.65143017 -0.89311563 -1.27410098]
 [-0.06115443  0.06451384  0.41011295 -0.57288249 -0.80133362  1.31203519]
 [ 1.27469887 -1.2143576   0.31371941 -1.44482142 -0.3689613  -0.76922658]
 [ 0.3926161   0.05729383  2.08997884  0.04197131 -0.04834072 -0.51315392]
 [-0.08458928 -1.21545008 -1.41293073 -1.48691055  0.38222486  0.937673  ]
 [ 1.77267804  0.87882801  0.33171912 -0.30603567  1.24026615 -0.21562684]
 [ 0.15592948  0.09805553  0.83209585  2.04520542 -0.31681392 -1.31283291]
 [-1.75445746  0.10209408 -1.36150208  0.48178488 -0.20832874 -0.09186351]
 [ 0.70268816  0.10365506  0.62123638  0.95411497  2.03781352 -0.48445122]]


Ejecutar función

In [8]:
svd_jacobi_aprox(a,10**-8,100)

(array([[-0.46799782,  0.18498431, -0.25271044,  0.50556768,  0.326373  ,
         -0.20802715],
        [-0.08941271, -0.2135826 ,  0.04309303, -0.58687765,  0.33463494,
         -0.01524759],
        [ 0.05794743, -0.57707707, -0.30085484,  0.40083168, -0.10133129,
          0.1271782 ],
        [ 0.32572214, -0.06794047, -0.43149003, -0.13970524,  0.33477343,
         -0.64719496],
        [-0.31248542, -0.4316653 ,  0.42201156,  0.11455876, -0.50069041,
         -0.16052517],
        [ 0.40961861, -0.14654734,  0.3483945 ,  0.33546987,  0.264984  ,
          0.31100118],
        [ 0.25440481,  0.44418061, -0.54604416,  0.04666071, -0.26277556,
          0.39326348],
        [-0.38855652,  0.37582276,  0.09308992,  0.07807463, -0.16848715,
         -0.09691082],
        [ 0.42708087,  0.19020835,  0.22645243,  0.29301151, -0.49072652,
         -0.48468409]]),
 array([4.59649174, 3.61220786, 2.81619661, 2.26909263, 1.67799987,
        1.1952781 ]),
 array([[ 0.61573379, -0.47281802, 